# 3x3 Rubiks' Cube Solver

Imports:

In [ ]:
from rubiks_cube import rubiks_cube as rc
import numpy as np
from copy import deepcopy as copy
from hashlib import sha1

We first make a solved cube:

In [ ]:
cube = rc.Cube()
cube.print()

In [ ]:
cube2 = copy(cube)

In [ ]:
cube2.turn_face('R')

In [ ]:
cube2.print()
cube.print()

We can get a one-hot representation of the cube like this:

In [ ]:
cube2.turn_face('R')
cube2.get_binary_array(one_hot=True).reshape(6, 8, 6)

## Detecting a G1 state

In [ ]:
def top_bottom_CO_Edge(bin_arr, relative_to=0):
    '''Whether the cube has all corners oriented and top/bottom edges oriented, relative to the top and bottom faces indexed as relative_to and relative_to + 3'''
    i = relative_to
    for face in (bin_arr[i*8:i*8+8], bin_arr[(i+3)*8:(i+3)*8+8]):
        for piece in face:
            if not piece[i] and not piece[i+3]:
                return False

    return True

In [ ]:
def mid_EO(bin_arr, relative_to=0):
    '''Whether the M slice edges are oriented, relative to the top and bottom faces indexed as relative_to and relative_to + 3'''
    if relative_to == 0:
        ind_to_check = ((1, 4), (1, 2), (4, 2), (4, 4))
        valid_faces = (1, 4)
    elif relative_to == 1:
        ind_to_check = ((0, 4), (0, 2), (3, 4), (3, 2))
        valid_faces = (0, 3)
    else:
        ind_to_check = ((4, 3), (4, 1), (1, 3), (1, 1))
        valid_faces = (1, 4)
    
    for (i, j) in ind_to_check:
        piece = bin_arr[i*8+j]
        if not piece[valid_faces[0]] and not piece[valid_faces[1]]:
            return False
        
    return True

In [ ]:
def is_in_G1(cube, relative_to=0):
    bin_arr = cube.get_binary_array(one_hot=True).reshape(-1, 6)
    return top_bottom_CO_Edge(bin_arr, relative_to=relative_to) and mid_EO(bin_arr, relative_to=relative_to)

In [ ]:
def is_solved(cube):
    return cube.is_solved()

## Brute force a first stage

We start with an arbitrary scramble:

In [ ]:
scramble = copy(cube)
scramble.scramble("R F L2")
# scramble.scramble("D B U' R' U R2 L' D F U2 F D2 R2 F2 R2 F2 B' D2 L2 U2 L'")
scramble.print()

In [ ]:
is_in_G1(scramble)

In [ ]:

face_move_map = {'U': 0, 'F': 1, 'R': 2, 'D': 3, 'B': 4, 'L': 5}

valid_moves = [('U', 0), ('U', 1), ('U', 2), 
    ('F', 0), ('F', 1), ('F', 2), 
    ('R', 0), ('R', 1), ('R', 2), 
    ('D', 0), ('D', 1), ('D', 2), 
    ('B', 0), ('B', 1), ('B', 2),
    ('L', 0), ('L', 1), ('L', 2)]

def next_valid_moves(prev_moves):
    if len(prev_moves) == 0:
        return valid_moves

    (last_face, _) = prev_moves[-1]
    last_ind = face_move_map[last_face]

    if len(prev_moves) > 1:
        (last2_face, _) = prev_moves[-2]
        last2_ind = face_move_map[last2_face]

        # WLOG, if you spin R then L, you don't want to do either again
        if abs(last2_ind - last_ind) == 3:
            ind = last_ind % 3
            return valid_moves[0:ind*3] + valid_moves[ind*3+3:(ind+3)*3] + valid_moves[(ind+3)*3+3:]

    return valid_moves[0:last_ind*3] + valid_moves[last_ind*3+3:]

all_180_moves = [('U', 2), 
    ('F', 2),
    ('R', 2), 
    ('D', 2), 
    ('B', 2), 
    ('L', 2)]

other_ways = [('U', 0), ('U', 1), 
    ('F', 0), ('F', 1), 
    ('R', 0), ('R', 1), 
    ('D', 0), ('D', 1), 
    ('B', 0), ('B', 1),
    ('L', 0), ('L', 1)]

def valid_g1_moves(relative_to=0):
    valid_g1s = all_180_moves + other_ways[relative_to*2:relative_to*2+2] + other_ways[(relative_to+3)*2:(relative_to+3)*2+2]
    return valid_g1s

In [ ]:
def turn_face(cube, move):
    next_cube = copy(cube)
    (face, way) = move
    next_cube.turn_face(face, way)
    return next_cube

In [ ]:
def solve_to_G1(scr_cube, try_relative_to=(0, 1, 2)):
    cubes_queue = [(scr_cube, [])]

    num_moves = 0
    i = 0

    while(True):
        (cube, prev_moves) = cubes_queue.pop(0)

        for rel in try_relative_to:
            if is_in_G1(cube, rel):
                print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
                return prev_moves
        
        if num_moves < len(prev_moves):
            num_moves = len(prev_moves)
            print("Now doing " + str(num_moves) + " moves, " + str(i) + " iters")
        
        for move in next_valid_moves(prev_moves):
            next_cube = turn_face(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

In [ ]:
g1_sol = solve_to_G1(scramble)
print(g1_sol)

This is _super_ slow :/

## Pruning tables: pt 2

We now explore solving a cube in a G1 state, using a pruning table that stores how close we are to a solution.

In [ ]:
def hash_cube(cube):
    return hash(str(cube.get_binary_array()))

In [ ]:
def calc_g1_movecount():
    G1_moves = valid_g1_moves(0)
    g1_sols = dict()
    q = [(copy(rc.Cube()), [])]
    max_moves = 5

    while(len(q) > 0):
        (cube, prev_moves) = q.pop(0)

        cube_hash = hash_cube(cube)

        if cube_hash not in g1_sols:
            g1_sols[cube_hash] = len(prev_moves)
            
            for move in G1_moves:
                next_cube = turn_face(cube, move)
                q.append((next_cube, prev_moves + [move]))
    
        if len(prev_moves) > max_moves:
            return g1_sols
    return g1_sols

g1_sols = calc_g1_movecount()

In [ ]:
len(g1_sols)

In [ ]:
def recover_sol(cube, g1_sols):
    G1_moves = valid_g1_moves(0)
    best_candidate = []
    shortest_len = 9001

    if cube.is_solved():
        return []

    
    hash_val = hash_cube(cube)
    if hash_val not in g1_sols:
        raise Error("Couldn't recover solution")
    
    moves_until_solved = g1_sols[hash_val]

    for move in G1_moves:
        cube_to_try = turn_face(cube, move)
        hash_trial = hash_cube(cube_to_try)
        if hash_trial in g1_sols:
            if g1_sols[hash_trial] < moves_until_solved:
                return [move] + recover_sol(cube_to_try, g1_sols)
    
    raise Error("Couldn't recover solution")

    return best_candidate

In [ ]:
test_cube = copy(cube)
test_cube.scramble("F2 U L2 R2")
print(g1_sols[hash_cube(test_cube)])
recover_sol(test_cube, g1_sols)

## Solve Cube from a Domino Reduction, using our pruning table

In [ ]:
def next_valid_g1_moves(g1_moves, prev_moves):
    if len(prev_moves) == 0:
        return g1_moves
    
    (last_face, _) = prev_moves[-1]
    
    return list(filter(lambda m: m[0] != last_face, g1_moves))

In [ ]:
def solve_from_domino(scr_cube, g1_sols):
    g1_moves = valid_g1_moves(0)
    cubes_queue = [(scr_cube, [])]

    num_moves = 0
    i = 0

    while(True):
        (cube, prev_moves) = cubes_queue.pop(0)
        hash_val = hash_cube(cube)

        if hash_val in g1_sols:
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return prev_moves + recover_sol(cube, g1_sols)
        
        if num_moves < len(prev_moves):
            num_moves = len(prev_moves)
            print("Now doing " + str(num_moves) + " moves, " + str(i) + " iters in")
        
        for move in next_valid_g1_moves(g1_moves, prev_moves):
            next_cube = turn_face(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

In [ ]:
test_cube = copy(cube)
test_cube.scramble("U2 B2 U2 F2 D2 F2 D2 R2 B2 U2 F2 R2 L2 U2 F2 U2 B2 F2 L2 U2 B2 D2 B2 F2 U2")
solve_from_domino(test_cube, g1_sols)

In [ ]:
test_cube2 = copy(test_cube)
test_cube2.scramble("U2 F2 R2 U' D L2 U' D' F2 L2")
test_cube2.print()
print(test_cube2.is_solved())

## Thistlethwaite's algorithm for splitting up solving to G1

Instead of immediately solving to <L2, R2, F2, B2, U, D>, we can solve to <L2, R2, F, B, U, D> and then <L2, R2, F2, B2, U, D> in two steps. We denote the first of the two steps EO, which is an (E)dge (O)rientation step.

In [ ]:
get_other_faces = {0: (1, 2), 1: (0, 2), 2: (0, 1)}

def check_eo(bin_arr, face_i, cant_have):
    for f in (face_i, face_i+3):
        for i in range(4):
            if bin_arr[f][i][cant_have] or bin_arr[f][i][cant_have+3]:
                return False
    
    return True

def cube_has_EO(cube, relative_to=2):
    # here, relative_to is defined the as axis with the 180 degree turns. 
    # in other words, relative_to=2 (index of right side) corresponds to the set
    # <L2, R2, F, B, U, D>. 
    # From here, we'd solve to G1 either relative_to 0 or relative_to 1
    # We have EO when none of the edges on one of the other_faces 
    # are on the other other_faces
    other_faces = get_other_faces[relative_to]
    bin_arr = cube.get_binary_array(one_hot=True).reshape(6, 8, 6)

    return check_eo(bin_arr, other_faces[0], other_faces[1]) and check_eo(bin_arr, other_faces[1], other_faces[0])

In [ ]:
cube_has_EO(cube5)

In [ ]:
def solve_to_EO(scr_cube, try_relative_to=(0, 1, 2), max_moves=9001):
    cubes_queue = [(scr_cube, [])]

    num_moves = 0
    i = 0

    candidates = []

    while(True):
        (cube, prev_moves) = cubes_queue.pop(0)

        for rel in try_relative_to:
            if cube_has_EO(cube, rel):
                candidates.append((prev_moves, rel))
        
        if num_moves < len(prev_moves):
            num_moves = len(prev_moves)
            print("Now doing " + str(num_moves) + " moves, " + str(i) + " iters")
        
        if num_moves > max_moves:
            print("Reached maximum number of moves: " + str(max_moves))
            return candidates
        
        for move in next_valid_moves(prev_moves):
            next_cube = turn_face(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

In [ ]:
get_valid_eo_moves = {
    0: [('U', 1), 
    ('F', 0), ('F', 1), ('F', 2), 
    ('R', 0), ('R', 1), ('R', 2), 
    ('D', 1),
    ('B', 0), ('B', 1), ('B', 2),
    ('L', 0), ('L', 1), ('L', 2)],
    1: [('U', 0), ('U', 1), ('U', 2), 
    ('F', 1),
    ('R', 0), ('R', 1), ('R', 2), 
    ('D', 0), ('D', 1), ('D', 2), 
    ('B', 1),
    ('L', 0), ('L', 1), ('L', 2)],
    2: [('U', 0), ('U', 1), ('U', 2), 
    ('F', 0), ('F', 1), ('F', 2), 
    ('R', 1),
    ('D', 0), ('D', 1), ('D', 2), 
    ('B', 0), ('B', 1), ('B', 2),
    ('L', 1)]
}

def next_valid_eo_moves(prev_moves, eo_relative_to):
    eo_valid_moves = get_valid_eo_moves[eo_relative_to]

    if len(prev_moves) == 0:
        return eo_valid_moves
    
    (last_face, _) = prev_moves[-1]
    
    return list(filter(lambda m: m[0] != last_face, eo_valid_moves))

In [683]:
def solve_to_G1(eo_cube, eo_relative_to, max_moves=9001):
    cubes_queue = [(eo_cube, [])]

    num_moves = 0
    i = 0

    candidates = []

    while(True):
        (cube, prev_moves) = cubes_queue.pop(0)

        for rel in get_other_faces[eo_relative_to]:
            if is_in_G1(cube, rel):
                candidates.append((prev_moves, rel))
        
        if num_moves < len(prev_moves):
            num_moves = len(prev_moves)
            # print("Now doing " + str(num_moves) + " moves, " + str(i) + " iters")

        if num_moves > max_moves:
            return candidates
        
        for move in next_valid_eo_moves(prev_moves, eo_relative_to):
            next_cube = turn_face(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

In [684]:
scramble = copy(cube)
scramble.scramble("D B U' R' U R2 L' D F U2 F D2 R2 F2 R2 F2 B' D2 L2 U2 L'")
scramble.print()

rubiks_cube: 

      o r w
      g b y
      o w w
      -----
g o b|y b r|b b o|g w y
r o r|y w o|b r o|y y b
y r w|r o o|b g g|r g b
      -----
      g w w
      g g w
      r y y



In [685]:
eo_sols = solve_to_EO(scramble, max_moves=4)
print(eo_sols)

Now doing 1 moves, 1 iters
Now doing 2 moves, 19 iters
Now doing 3 moves, 289 iters
Now doing 4 moves, 4177 iters
Now doing 5 moves, 60553 iters
Reached maximum number of moves: 4
[([('F', 0), ('R', 1), ('F', 0)], 1), ([('F', 0), ('R', 1), ('F', 1)], 1), ([('F', 1), ('D', 0), ('R', 0)], 1), ([('F', 1), ('D', 0), ('R', 1)], 1), ([('F', 2), ('R', 1), ('F', 0)], 1), ([('F', 2), ('R', 1), ('F', 1)], 1), ([('U', 0), ('R', 1), ('L', 0), ('F', 0)], 1), ([('U', 0), ('R', 1), ('L', 0), ('F', 1)], 1), ([('U', 0), ('D', 0), ('R', 0), ('F', 0)], 2), ([('U', 0), ('D', 0), ('R', 0), ('F', 1)], 2), ([('U', 0), ('D', 0), ('R', 1), ('F', 0)], 2), ([('U', 0), ('D', 0), ('R', 1), ('F', 1)], 2), ([('U', 0), ('L', 0), ('F', 0), ('U', 0)], 1), ([('U', 0), ('L', 0), ('F', 0), ('U', 1)], 1), ([('U', 0), ('L', 0), ('R', 1), ('F', 0)], 1), ([('U', 0), ('L', 0), ('R', 1), ('F', 1)], 1), ([('U', 2), ('D', 1), ('L', 0), ('B', 0)], 1), ([('U', 2), ('D', 1), ('L', 0), ('B', 1)], 1), ([('U', 2), ('D', 2), ('L', 1), (

In [686]:
eo_state = copy(scramble)
eo_sol = eo_sols[0]
for (face, way) in eo_sol[0]:
    eo_state.turn_face(face, way)
solve_to_G1(eo_state, eo_sol[1], max_moves=4)

[]

In [695]:
len(eo_sols)

218

In [698]:
def find_g1_from_eo_candidates(eo_candidates, max_move_range):
    g1_candidates = []
    for max_moves in max_move_range:
        print("Trying to find G1's for " + str(max_moves) + " moves")
        i = 0
        for (eo_sol, eo_rel_to) in eo_sols:
            i = i+1
            print("Candidate " + str(i) + ": " + str(eo_sol))
            for move in eo_sol:
                eo_state.turn_face(move[0], move[1])
            g1_sols = solve_to_G1(eo_state, eo_rel_to, max_moves=(max_moves-len(eo_sol)))

            if len(g1_sols) > 0:
                print("Found a G1 solution! len(EO):" + str(len(eo_sol)) + ", len(G1):" + str(len(g1_sols[0])))
                for g1_sol in g1_sols:
                    g1_candidates.append(((eo_sol, eo_rel_to), g1_sol))
        
        if len(g1_candidates) > 0:
            return g1_candidates
    return g1_candidates

In [699]:
g1_sol = find_g1_from_eo_candidates(eo_sols, range(6, 7))

Trying to find G1's for 6 moves
Candidate 1: [('F', 0), ('R', 1), ('F', 0)]
Candidate 2: [('F', 0), ('R', 1), ('F', 1)]
Candidate 3: [('F', 1), ('D', 0), ('R', 0)]
Candidate 4: [('F', 1), ('D', 0), ('R', 1)]
Candidate 5: [('F', 2), ('R', 1), ('F', 0)]
Candidate 6: [('F', 2), ('R', 1), ('F', 1)]
Candidate 7: [('U', 0), ('R', 1), ('L', 0), ('F', 0)]
Candidate 8: [('U', 0), ('R', 1), ('L', 0), ('F', 1)]
Candidate 9: [('U', 0), ('D', 0), ('R', 0), ('F', 0)]
Candidate 10: [('U', 0), ('D', 0), ('R', 0), ('F', 1)]
Candidate 11: [('U', 0), ('D', 0), ('R', 1), ('F', 0)]
Candidate 12: [('U', 0), ('D', 0), ('R', 1), ('F', 1)]
Candidate 13: [('U', 0), ('L', 0), ('F', 0), ('U', 0)]
Candidate 14: [('U', 0), ('L', 0), ('F', 0), ('U', 1)]
Candidate 15: [('U', 0), ('L', 0), ('R', 1), ('F', 0)]
Candidate 16: [('U', 0), ('L', 0), ('R', 1), ('F', 1)]
Candidate 17: [('U', 2), ('D', 1), ('L', 0), ('B', 0)]
Candidate 18: [('U', 2), ('D', 1), ('L', 0), ('B', 1)]
Candidate 19: [('U', 2), ('D', 2), ('L', 1), ('

In [700]:
g1_sol = find_g1_from_eo_candidates(eo_sols, range(7, 8))

Trying to find G1's for 7 moves
Candidate 1: [('F', 0), ('R', 1), ('F', 0)]
Candidate 2: [('F', 0), ('R', 1), ('F', 1)]
Candidate 3: [('F', 1), ('D', 0), ('R', 0)]
Candidate 4: [('F', 1), ('D', 0), ('R', 1)]
Candidate 5: [('F', 2), ('R', 1), ('F', 0)]
Candidate 6: [('F', 2), ('R', 1), ('F', 1)]
Candidate 7: [('U', 0), ('R', 1), ('L', 0), ('F', 0)]
Candidate 8: [('U', 0), ('R', 1), ('L', 0), ('F', 1)]
Candidate 9: [('U', 0), ('D', 0), ('R', 0), ('F', 0)]
Candidate 10: [('U', 0), ('D', 0), ('R', 0), ('F', 1)]
Candidate 11: [('U', 0), ('D', 0), ('R', 1), ('F', 0)]
Candidate 12: [('U', 0), ('D', 0), ('R', 1), ('F', 1)]
Candidate 13: [('U', 0), ('L', 0), ('F', 0), ('U', 0)]
Candidate 14: [('U', 0), ('L', 0), ('F', 0), ('U', 1)]
Candidate 15: [('U', 0), ('L', 0), ('R', 1), ('F', 0)]
Candidate 16: [('U', 0), ('L', 0), ('R', 1), ('F', 1)]
Candidate 17: [('U', 2), ('D', 1), ('L', 0), ('B', 0)]
Candidate 18: [('U', 2), ('D', 1), ('L', 0), ('B', 1)]
Candidate 19: [('U', 2), ('D', 2), ('L', 1), ('

In [701]:
g1_sol = find_g1_from_eo_candidates(eo_sols, range(7, 8))

Trying to find G1's for 7 moves
Candidate 1: [('F', 0), ('R', 1), ('F', 0)]
Candidate 2: [('F', 0), ('R', 1), ('F', 1)]
Candidate 3: [('F', 1), ('D', 0), ('R', 0)]
Candidate 4: [('F', 1), ('D', 0), ('R', 1)]
Candidate 5: [('F', 2), ('R', 1), ('F', 0)]
Candidate 6: [('F', 2), ('R', 1), ('F', 1)]
Candidate 7: [('U', 0), ('R', 1), ('L', 0), ('F', 0)]
Candidate 8: [('U', 0), ('R', 1), ('L', 0), ('F', 1)]
Candidate 9: [('U', 0), ('D', 0), ('R', 0), ('F', 0)]
Candidate 10: [('U', 0), ('D', 0), ('R', 0), ('F', 1)]
Candidate 11: [('U', 0), ('D', 0), ('R', 1), ('F', 0)]
Candidate 12: [('U', 0), ('D', 0), ('R', 1), ('F', 1)]
Candidate 13: [('U', 0), ('L', 0), ('F', 0), ('U', 0)]
Candidate 14: [('U', 0), ('L', 0), ('F', 0), ('U', 1)]
Candidate 15: [('U', 0), ('L', 0), ('R', 1), ('F', 0)]
Candidate 16: [('U', 0), ('L', 0), ('R', 1), ('F', 1)]
Candidate 17: [('U', 2), ('D', 1), ('L', 0), ('B', 0)]
Candidate 18: [('U', 2), ('D', 1), ('L', 0), ('B', 1)]
Candidate 19: [('U', 2), ('D', 2), ('L', 1), ('

Oof... that's slow.

## Heurstic for G1: corner orientation

To be in G1, all the corners have to be oriented. We can use this as a heuristic to speed up our G1 solution finder.
We know that any solution that solves G1 must also orient the corners. Thus, solving to G1 must take at least the number of moves to orient the corners.